In [55]:
# -*- coding: utf-8 -*- 
# use python 3 

import sys
import re
import rsa
import http.cookiejar  # cookielib in python 2
import base64
import json
import urllib
import urllib.parse
import binascii
import codecs
import pandas as pd
from lxml import etree
import os
import time
import datetime
import random
from urllib.parse import urlencode, quote

import urllib.error
import urllib.request
from DecryptLogin import login  #爬虫库
import logging
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [56]:
sys.setrecursionlimit(1000000)

#local_disk = False
local_disk = True

In [95]:
def download_one_user(user_id, start_time, end_time, headers, session):
    
    weibos=[]
    over50=0
    


#     if local_disk:
#         dirname = "user-post-results/" + user_id + "/"
#     else:
#         # dirname = "/Volumes/Seagate/weibo/keyword-search-results/" + keyword + "/"
#         # this is the Ubuntu data storage set
#         dirname = "/media/han/external/home/han/Grid/keyword-search-results/" + keyword + "/"

#     if not os.path.exists(dirname):                   #判断是否存在文件夹如果不存在则创建为文件夹
#         os.makedirs(dirname)            #makedirs 创建文件时如果路径不存在会创建这个路径

        
    now = datetime.datetime.now()
    # year = now.year
    crawler_time = now.strftime('%Y-%m-%d-%H-%M-%S')
    if start_time >= end_time:
        sys.exit(0)

## page number

    url = f' https://weibo.cn/{user_id}/profile?hasori=0&haspic=0&starttime={start_time}&endtime={end_time}&advancedfilter=1&page=1'
    res = session.get(url, headers=headers)
    selector = etree.HTML(res.content)
    num_wb_pages = selector.xpath("//input[@name='mp']")
    page_num = int(num_wb_pages[0].attrib['value']) if len(num_wb_pages) > 0 else 1
    
    page_block_size = random.randint(1, 5)
    page_block_count = 0
    
    
    print ("page num of this time", page_num)
    logging.info("page num of this time %s" % page_num)
    logging.info("begin downloading with page number {} ".format(page_num))  

    if (page_num>50):
        return over50,weibos
#         over50=1

## then iterate to save all pages, max=50

    page_num = min(page_num, 50)
    

    
    for page in range(1, page_num+1):
        url = f' https://weibo.cn/{user_id}/profile?hasori=0&haspic=0&starttime={start_time}&endtime={end_time}&advancedfilter=1&page={page}'
        #hasori:是否原创， haspic：包含图片
        
        res = session.get(url, headers=headers)
        selector = etree.HTML(res.content)
        contents = selector.xpath("//div[@class='c']")
        if contents[0].xpath("div/span[@class='ctt']"):
            for i in range(0, len(contents)-2):
                content = contents[i]
                
                weibo_info = parseWeiboContent(content)
                
                weibo=[user_id]+weibo_info
                
                weibos.append(weibo)
                
                    
            page_block_count += 1
            if page_block_count % page_block_size == 0:
                time.sleep(random.randint(6, 12))
                page_block_size = random.randint(1, 5)
                page_block_count = 0
        logging.info('用户%s的所有微博数据下载完毕' % user_id)
    
    return over50,weibos


    


In [96]:
'''处理不同类型的微博'''
    
def parseWeiboContent(content):
    
    # 是否为原创微博
    is_ori = False if len(content.xpath("div/span[@class='cmt']")) > 3 else True
    # 微博ID
    weibo_id = content.xpath('@id')[0][2:]
    # 微博文本
    if is_ori:
        if u'全文' in content.xpath('div//a/text()'):
            url = f'https://weibo.cn/comment/{weibo_id}'
            res = session.get(url, headers=headers)
            selector = etree.HTML(res.content)
            weibo_text_tmp = selector.xpath("//div[@class='c']")[1]
            weibo_text = weibo_text_tmp.xpath('string(.)').replace(u'\u200b', '').encode('utf-8', 'ignore').decode('utf-8')
            weibo_text = weibo_text[weibo_text.find(':')+1: weibo_text.rfind(weibo_text_tmp.xpath("//span[@class='ct']/text()")[0])]
        else:
            weibo_text = content.xpath('string(.)').replace(u'\u200b', '').encode('utf-8', 'ignore').decode('utf-8')
            weibo_text = weibo_text[:weibo_text.rfind(u'赞')]
    else:
        if u'全文' in content.xpath('div//a/text()'):
            url = f'https://weibo.cn/comment/{weibo_id}'
            res = session.get(url, headers=headers)
            selector = etree.HTML(res.content)
            weibo_text_tmp = selector.xpath("//div[@class='c']")[1]
            weibo_text = weibo_text_tmp.xpath('string(.)').replace(u'\u200b', '').encode('utf-8', 'ignore').decode('utf-8')
            weibo_text = weibo_text[weibo_text.find(':')+1: weibo_text.rfind(weibo_text_tmp.xpath("//span[@class='ct']/text()")[0])]
            weibo_text = weibo_text[:weibo_text.rfind(u'原文转发')]
        else:
            weibo_text = content.xpath('string(.)').replace(u'\u200b', '').encode('utf-8', 'ignore').decode('utf-8')
            weibo_text = weibo_text[weibo_text.find(':')+1: weibo_text.rfind(u'赞')][:weibo_text.rfind(u'赞')]
    # 微博发布的时间
    publish_time = content.xpath("div/span[@class='ct']")[0]
    publish_time = publish_time.xpath('string(.)').replace(u'\u200b', '').encode('utf-8', 'ignore').decode('utf-8')
    publish_time = publish_time.split(u'来自')[0]
    if u'刚刚' in publish_time:
        publish_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    elif u'分钟' in publish_time:
        passed_minutes = publish_time[:publish_time.find(u'分钟')]
        publish_time = (datetime.datetime.now() - datetime.timedelta(int(passed_minutes))).strftime('%Y-%m-%d %H:%M')
    elif u'今天' in publish_time:
        today = datetime.datetime.now().strftime('%Y-%m-%d')
        publish_time = today + ' ' + publish_time[3:]
        if len(publish_time) > 16: publish_time = publish_time[:16]
    elif u'月' in publish_time:
        year = datetime.datetime.now().strftime('%Y')
        month = publish_time[0: 2]
        day = publish_time[3: 5]
        publish_time = year + '-' + month + '-' + day + ' ' + publish_time[7: 12]
    else:
        publish_time = publish_time[:16]

    # 整合数据
    weibo_info= [weibo_id, publish_time, weibo_text]
    # 返回提取的数据
    return weibo_info


In [86]:

username = ""; password =''

client = login.Client()
lg=client.weibo(reload_history=True) #reload the login history
infos_return,session = lg.login(username, password, 'mobile') #enter SMS code


Welcome to use DecryptLogin!
You can visit https://github.com/CharlesPikachu/DecryptLogin for more details.
[INFO]: Resume 1029200273xx@sina.com's session and infos from C:\Users\10292\anaconda3\lib\site-packages\DecryptLogin\modules\clients\weibo.pkl


In [97]:
## read nationalistic users' id

user_id = pd.read_csv('Chongqing & Shanghai.csv',header=None)[0].apply(str).values.tolist()[1:]


In [98]:
len(user_id)

59

In [100]:
## search userid and download posts



logging.info("START new user-----------------------------------------------------------------")

headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
    }

logging.basicConfig(filename='download-weibo-user-posts-dynamic-%s.log'%user_id,
format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S',level=logging.INFO)

start_time = "20220201"
end_time ="20220901"

weibos=[]
over50_ids=[]

for u in range(len(user_id[:2])):
    if (u % 20==0):
        print ("downloaded user ", u)
    
    over50,weibo=download_one_user(user_id[u], start_time, end_time, headers=headers, session = session)
    weibos=weibos+weibo
#     if (over50==1):
#         over50_ids.append(user_id[u])


downloaded user  0
page num of this time 11
page num of this time 29


In [101]:
import pandas as pd
wb=pd.DataFrame(weibos,columns=['user_id','weibo_id','post time','content'])
wb.to_csv("full_weibo.csv", encoding='utf_8_sig')

## Cope with users with 50 or more pages

In [131]:
def download_one_user_over50(user_id, start_time, end_time, headers, session):
    
    weibos=[]
    
        
    now = datetime.datetime.now()
    # year = now.year
    crawler_time = now.strftime('%Y-%m-%d-%H-%M-%S')
    if start_time >= end_time:
        sys.exit(0)

## page number

    url = f' https://weibo.cn/{user_id}/profile?hasori=0&haspic=0&starttime={start_time}&endtime={end_time}&advancedfilter=1&page=1'
    print(url)
    res = session.get(url, headers=headers)
    selector = etree.HTML(res.content)
    num_wb_pages = selector.xpath("//input[@name='mp']")
    page_num = int(num_wb_pages[0].attrib['value']) if len(num_wb_pages) > 0 else 1
    
    page_block_size = random.randint(1, 5)
    page_block_count = 0
    
    
    print ("page num of this time", page_num)
    logging.info("page num of this time %s" % page_num)
    logging.info("begin downloading with page number {} ".format(page_num))  



## then iterate to save all pages, max=50

    page_num = min(page_num, 50)
    

    
    for page in range(1, page_num+1):
        url = f' https://weibo.cn/{user_id}/profile?hasori=0&haspic=0&starttime={start_time}&endtime={end_time}&advancedfilter=1&page={page}'
        #hasori:是否原创， haspic：包含图片
        
        res = session.get(url, headers=headers)
        selector = etree.HTML(res.content)
        contents = selector.xpath("//div[@class='c']")
        if contents[0].xpath("div/span[@class='ctt']"):
            for i in range(0, len(contents)-2):
                content = contents[i]
                
                weibo_info = parseWeiboContent(content)
                
                weibo=[user_id]+weibo_info
                
                weibos.append(weibo)
                
                    
            page_block_count += 1
            if page_block_count % page_block_size == 0:
                time.sleep(random.randint(6, 12))
                page_block_size = random.randint(1, 5)
                page_block_count = 0
        logging.info('用户%s的所有微博数据下载完毕' % user_id)
    
    return weibos


    


In [132]:
over50_id = pd.read_csv('user_CQ_SH_over50pages.csv',header=None)[0].apply(str).values.tolist()
over50_id

['3215131980',
 '7585295356',
 '2690549404',
 '1161362515',
 '5287998142',
 '2528626364',
 '6103005527',
 '5303226875',
 '7295943862',
 '2019665801']

In [133]:
over50_id_d=['3215131980',
 '2690549404',
 '1161362515',
 '5287998142',
 '2528626364',
 '6103005527',
 '5303226875',
 '7295943862']

In [ ]:
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
    }

In [130]:
start_time = "20220201"
end_time ="20220501"

weibos=[]

for u in range(len(over50_id_d)):
    print(over50_id_d[u])
    weibo=download_one_user_over50(over50_id_d[u], start_time, end_time, headers=headers, session = session)


    weibos=weibos+weibo
wb=pd.DataFrame(weibos,columns=['user_id','weibo_id','post time','content'])
wb.to_csv("full_weibo_over50.csv", mode='a',index=False,encoding='utf_8_sig')

3215131980
 https://weibo.cn/3215131980/profile?hasori=0&haspic=0&starttime=20220201&endtime=20220501&advancedfilter=1&page=1
page num of this time 43
2690549404
 https://weibo.cn/2690549404/profile?hasori=0&haspic=0&starttime=20220201&endtime=20220501&advancedfilter=1&page=1
page num of this time 17
1161362515
 https://weibo.cn/1161362515/profile?hasori=0&haspic=0&starttime=20220201&endtime=20220501&advancedfilter=1&page=1
page num of this time 32
5287998142
 https://weibo.cn/5287998142/profile?hasori=0&haspic=0&starttime=20220201&endtime=20220501&advancedfilter=1&page=1
page num of this time 31
2528626364
 https://weibo.cn/2528626364/profile?hasori=0&haspic=0&starttime=20220201&endtime=20220501&advancedfilter=1&page=1
page num of this time 1
6103005527
 https://weibo.cn/6103005527/profile?hasori=0&haspic=0&starttime=20220201&endtime=20220501&advancedfilter=1&page=1
page num of this time 33
5303226875
 https://weibo.cn/5303226875/profile?hasori=0&haspic=0&starttime=20220201&endtime=20

In [134]:
start_time = "20220502"
end_time ="20220701"

weibos=[]

for u in range(len(over50_id_d)):
    print(over50_id_d[u])
    weibo=download_one_user_over50(over50_id_d[u], start_time, end_time, headers=headers, session = session)


    weibos=weibos+weibo
wb=pd.DataFrame(weibos,columns=['user_id','weibo_id','post time','content'])
wb.to_csv("full_weibo_over50.csv", mode='a',index=False,header=False,encoding='utf_8_sig')

2690549404
 https://weibo.cn/2690549404/profile?hasori=0&haspic=0&starttime=20220502&endtime=20220701&advancedfilter=1&page=1
page num of this time 22


In [135]:
start_time = "20220702"
end_time ="20220801"

weibos=[]

for u in range(len(over50_id_d)):
    print(over50_id_d[u])
    weibo=download_one_user_over50(over50_id_d[u], start_time, end_time, headers=headers, session = session)
    weibos=weibos+weibo
wb=pd.DataFrame(weibos,columns=['user_id','weibo_id','post time','content'])
wb.to_csv("full_weibo_over50.csv", mode='a',index=False,header=False,encoding='utf_8_sig')
    

2690549404
 https://weibo.cn/2690549404/profile?hasori=0&haspic=0&starttime=20220702&endtime=20220801&advancedfilter=1&page=1
page num of this time 9


In [ ]:
# abnormally active

# 7585295356
#  https://weibo.cn/7585295356/profile?hasori=0&haspic=0&starttime=20220201&endtime=20220901&advancedfilter=1&page=1
# page num of this time 224

# 2019665801
#  https://weibo.cn/2019665801/profile?hasori=0&haspic=0&starttime=20220201&endtime=20220901&advancedfilter=1&page=1
# page num of this time 449



In [93]:
over50_ids

['5303226875',
 '7295943862',
 '5287998142',
 '2528626364',
 '7585295356',
 '3215131980',
 '1161362515',
 '6103005527',
 '2019665801',
 '2690549404']

In [94]:
df=pd.DataFrame({'uid':list(set(over50_ids))})
df.to_csv('user_CQ_SH_over50pages.csv',mode='a', index=False,header=False)

In [76]:
import pandas as pd
wb=pd.DataFrame(weibos,columns=['user_id','weibo_id','post time','content'])


In [77]:
wb

,user_id,weibo_id,post time,content
0,7630138756,M3I9smspz,2022-09-01 11:12,这才是开学第一课该有的样子。 -四分三十三秒-的微博视频 赞[718] 原文转发[43] ...
1,7630138756,M3suLg8qW,2022-08-30 19:20,#解放军在黄河上架设254米浮桥#上视频！我军某新型轮式两栖自行门桥。 -四分三十三秒-的微...
2,7630138756,M39Jbs8zf,2022-08-28 19:34,车塞满了爱，眼里流着泪。 -四分三十三秒-的微博视频 赞[851] 原文转发[51] 原文...
3,7630138756,M38tQeDe7,2022-08-28 16:23,#山城夹道长队送别灭火英雄#这就是外国人永远不会懂得场景。 [组图共4张] 原图 赞[60...
4,7630138756,M38cSkOcU,2022-08-28 15:41,抱歉，作者已设置仅展示半年内微博，此微博已不可见。 赞[0] 原文转发[0] 原文评论[0...
...,...,...,...,...
223,6510420346,LhLNB791y,2022-03-01 23:29,【#陕西针对铁笼女事件成立调查组#】据陕西榆林市公安局通报，2022年3月1日0时许，微信公...
224,6510420346,Lhxe4CdQN,2022-02-28 10:23,有件有意思的事情，大家可以都仔细想一想。为什么在国际关系中，只有中国人、只有中国人需要考虑自...
225,6510420346,Lhp6DckXM,2022-02-27 13:43,突然想到一个思考的点，就现在国内舆论平台及舆论节奏风向该怎么整治的问题，，，为什么说需要整治...
226,6510420346,LgVp07moN,2022-02-24 10:06,【芙蓉鲜蔬汤】多种蔬菜煮成，含有大量的维生素蛋白质，营养丰富。 #一起学做菜# [组图共...


In [40]:
wb.to_csv("full_weibo.csv", encoding='utf_8_sig')

In [41]:
 pd.read_csv('full_weibo.csv')

,Unnamed: 0,user_id,weibo_id,post time,content
0,0,7630138756,M3I9smspz,2022-09-01 11:12,这才是开学第一课该有的样子。 -四分三十三秒-的微博视频 赞[718] 原文转发[43] ...
1,1,7290362662,M2zTCDL8A,2022-08-25 00:21,超话粉丝大咖
2,2,6510420346,M38tQzxAX,2022-08-28 16:23,美国海军表示，两艘军舰周日（28日）通过台湾海峡，这是自美国众议院议长佩洛西访台加剧美中紧张...
3,3,2786215997,M3xxpvD2l,2022-08-31 08:11,#一条plog告别八月#咋天，母亲大人来电，告诉我，亲戚中最大的长辈去世了……有时候感觉，为...
4,4,1764041015,M3zpl1rvx,2022-08-31 12:56,抱歉，作者已设置仅展示半年内微博，此微博已不可见。 赞[0] 原文转发[0] 原文评论[0...
...,...,...,...,...,...
721,721,5892119506,M2b4Ms3Ay,2022-08-22 09:09,抱歉，作者已设置仅展示半年内微博，此微博已不可见。 赞[0] 原文转发[0] 原文评论[0...
722,722,6532313666,M0iFdsLRq,2022-08-10 00:48,#南昌每5套房就有1套空着#这事大家是默认这个机构说的都是对的？我是懒得查数据，南昌的房子数...
723,723,6749160035,M3MMGb9lt,2022-09-01 23:00,成都的情况再次证明，常态化早发现早控制疫情的能力最重要。#鸿儒计划##胡侃# 胡锡进的微博视...
724,724,2690549404,M3GY6d2r8,2022-09-01 08:11,今天全市中小学开学啦！早高峰交通又堵了。上海部分高等院校还未开，甚至国庆节后才开，情何以以堪...


In [31]:
##github link
#https://github.com/Python3Spiders/WeiboSuperSpider/blob/master/%E6%97%A0%20GUI%20%E5%8A%9F%E8%83%BD%E7%8B%AC%E7%AB%8B%E7%89%88/WeiboUserInfoSpider.py


import requests

import pandas as pd

from time import sleep

import json

headers = {
    'authority': 'weibo.com',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://weibo.com/1192329374/KnnG78Yf3?filter=hot&root_comment_id=0&type=comment',
    'accept-language': 'zh-CN,zh;q=0.9,en-CN;q=0.8,en;q=0.7,es-MX;q=0.6,es;q=0.5',
    'cookie': 'SINAGLOBAL=3804619408460.8784.1655050413008; UOR=,,www.google.com; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5KSyp6BSqzN84wimuLmIMa5JpX5KMhUgL.FoeNeKzXeh.f1KB2dJLoI7vZdXDaUPzLxKnLBK2L12et; SCF=Aij1M7mMZab6kJ8N2Hanc-QoiuIalQYFetSYk8Ivx0150vfrRXjwVrsDF5_8_5RYmD9Jg_KTH3OjwuPV5-UVDu4.; SUB=_2A25O5aC5DeRhGeVJ6lAV8CfJwjiIHXVtkpVxrDV8PUNbmtAKLWXDkW9NT-1CQDP6Eq45tYQXDuvU6DYiMN81KavW; XSRF-TOKEN=vjnTjDzLAsmCV-S1b0RzaL5a; PC_TOKEN=ba2fa8f64e; _s_tentry=-; Apache=447948548589.99365.1675776230646; ULV=1675776230700:17:2:1:447948548589.99365.1675776230646:1675475774825; wb_view_log_3712409594=1920*10801; webim_unReadCount={"time":1675776355088,"dm_pub_total":0,"chat_group_client":0,"chat_group_notice":0,"allcountNum":0,"msgbox":0}; WBPSESS=E50Btn0ZG3GPfmwGhfl0dtFJJbiH4NvgPcN0AWq5UV5xsJE_azbWK6CO4JaksmYS1zEb5UGONRfeB4XsO4Jb9yAI96dskOKWmzSxZeQO16ZHsKgy6Cdcbg1kJ-XJC72d-WMI3KFfkvdQLdv_jeZvRg=='
}

#cookie 可能要变：   # https://s.weibo.com/user?q=%E6%B5%8B%E8%AF%95&Refer=weibo_user cookie = '复制上面接口登录后的 cookie'
# https://blog.csdn.net/qq_36652557/article/details/122215966 自动 get cookie

def parseUid(uid):
    response = requests.get(url=f'https://weibo.com/ajax/profile/info?custom={uid}', headers=headers)
    try:
        return response.json()['data']['user']['id']
    except:
        return None


def getUserInfo(uid):
    try:
        uid = int(uid)
    except:
        # 说明是 xiena 这样的英文串
        uid = parseUid(uid)
        if not uid:
            return None
    response = requests.get(url=f'https://weibo.com/ajax/profile/detail?uid={uid}', headers=headers)
    if response.status_code == 400:
        return {
            'errorMsg': '用户可能注销或者封号',
            'location': None,
            'user_link': f'https://weibo.com/{uid}'
        }
    resp_json = response.json().get('data', None)
    if not resp_json:
        return None
    sunshine_credit = resp_json.get('sunshine_credit', None)
    if sunshine_credit:
        sunshine_credit_level = sunshine_credit.get('level', None)
    else:
        sunshine_credit_level = None
    education = resp_json.get('education', None)
    if education:
        school = education.get('school', None)
    else:
        school = None

    location = resp_json.get('ip_location', None)[5:]
    gender = resp_json.get('gender', None)

    birthday = resp_json.get('birthday', None)
    created_at = resp_json.get('created_at', None)
    description = resp_json.get('description', None)
    # 我关注的人中有多少人关注 ta
    followers = resp_json.get('followers', None)
    if followers:
        followers_num = followers.get('total_number', None)
    else:
        followers_num = None
    return {
        'user_id':uid,
        'sunshine_credit_level': sunshine_credit_level,
        'school': school,
        'location': location,
        'gender': gender,
        'birthday': birthday,
        'created_at': created_at,
        'description': description,
        'followers_num': followers_num
    }




if __name__ == '__main__':
    user_info = getUserInfo(uid='3712409594')
    print(user_info)

{'user_id': 3712409594, 'sunshine_credit_level': '信用较好', 'school': None, 'location': '中国香港', 'gender': 'f', 'birthday': '2021-03-24', 'created_at': '2013-08-10 19:46:21', 'description': '开心🥳', 'followers_num': None}
